## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
import pandas as pd
bank_loan = pd.read_csv('/datasets/credit_scoring_eng.csv')
print(bank_loan.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


### Conclusion

To look at the bank_loan information call the info() method to see the columns, the number of entries, and the data types.

To see an example of the bank_loan table use the head() method to print the first couple of rows of the table.

### Step 2. Data preprocessing

### Processing missing values

In [14]:
bank_loan.head()

bank_loan['days_employed'] = bank_loan['days_employed'].abs()

bank_loan.info()


bank_loan[bank_loan['days_employed'].isnull()].count()
bank_loan[bank_loan['total_income'].isnull()].count()

#Replace missing days_employed with days_employed avg
days_employed_avg = bank_loan['days_employed'].mean()
bank_loan['days_employed'] = bank_loan['days_employed'].fillna(days_employed_avg)
bank_loan[bank_loan['days_employed'].isnull()].count()

#Replace missing total_income with total_income avg
total_income_avg = bank_loan['total_income'].mean()
bank_loan['total_income'] = bank_loan['total_income'].fillna(total_income_avg)
bank_loan[bank_loan['total_income'].isnull()].count()







<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 16 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
purpose_group       21525 non-null object
education_group     21525 non-null object
age_group           21525 non-null object
income_group        21525 non-null object
dtypes: int64(7), object(9)
memory usage: 2.6+ MB


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
purpose_group       0
education_group     0
age_group           0
income_group        0
dtype: int64

### Conclusion

After looking at the bank_loan information using the info() method, it looks as if the days employed and total income columns have missing and abnormal values.

In the first couple of rows it is seen that the days employed column contains negative values. There can't be a negative number of days employed. The days employed column needs to be changed to all positive float values using the absolute value function. 

The missing values for the days employed column most likely comes from user input where the user didn't know when they started their job or how many days they have been working for.

The missing values for the total income column is also most likely from user input where the user didn't know how much total income they made or forgot to input the value all together. 

Using the isnull() method, it was discoverd that:
The days employed is missing values in 2174 out of the 21525 rows
The total income is missing values in 2174 out of the 21525 rows

The missing values of days employed were replaced with the average of all days employed
The missing values of total income were replaced with the average of all total income

### Data type replacement

In [3]:
bank_loan['days_employed'] = bank_loan['days_employed'].astype('int')


bank_loan['total_income'] = bank_loan['total_income'].astype('int')
bank_loan.info(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Conclusion

To change the data types of days_employed and total_income, use the astype() method to select the data type int. I used this method because the to_numeric() method is used to convert the values of a column to a floating point number. 

### Processing duplicates

In [4]:
bank_loan['education'].value_counts()
bank_loan['education'] = bank_loan['education'].str.lower()
bank_loan['education'].value_counts()          

bank_loan['children'].value_counts()

bank_loan['children'] = bank_loan['children'].replace(-1, 1)
bank_loan['children'] = bank_loan['children'].replace(20, 2)

bank_loan['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Conclusion

There are duplicate values in the education column. The main reason for the duplicates in the education column is case sensitivity. Many of the education types were the same but some types had uppercase letters while others had lowercase letters.

Since the education column is a string, to delete duplicates I used the str.lower() method to get rid of all of the  duplicates in education types.

To confirm that the duplicates have been changed call the value_counts() method for the education column to look at the counts of unique values for each education type. 

There are also incorrect values in the children column. The main reason for the duplicate values is that the information was input incorrectly by the user. There are 47 values that equal -1 and 76 values that equal 20. The negative 1s need to be changed to positive and the values that equal 20 mostly like accidently put an extra 0, so these values are changed to 2. 

Use the replace() method to change the values to the correct number of children. Negative 1 was replaced with 1, and 20 was replaced with 2.


### Categorizing Data

In [5]:
# Method to group loan type by purpose
def loan_group_purpose(row):
    purpose = row['purpose']
    
    if 'house' in purpose:
        return 'real estate loan purpose'
    if 'housing' in purpose:
        return 'real estate loan purpose'
    if 'wedding' in purpose:
        return 'wedding loan purpose'
    if 'real estate' in purpose:
        return 'real estate loan purpose'
    if 'property' in purpose:
        return 'real estate loan purpose'
    if 'car' in purpose:
        return 'car loan purpose'
    if 'university' in purpose:
        return 'school loan purpose'
    if 'education' in purpose:
        return 'school loan purpose'
    
    return 'purpose unidentified'


bank_loan['purpose_group'] = bank_loan.apply(loan_group_purpose, axis=1)
bank_loan['purpose_group'].value_counts()


# Method to group loan type by education level
def loan_group_education(row):
    education = row['education']
    
    if 'secondary education' in education:
        return 'K-12 education'
    if 'bachelor\'s degree' in education:
        return 'College education'
    if 'some college' in education:
        return 'College education'
    if 'primary education' in education:
        return 'K-12 education'
    if 'graduate degree' in education:
        return 'Graduate education'
    
    return 'education unidentified'

bank_loan['education_group'] = bank_loan.apply(loan_group_education, axis=1)
bank_loan['education_group'].value_counts()

# Value count to see the male to female ratio
bank_loan['gender'].value_counts()

# Method to group loan type by age level
bank_loan['dob_years'].value_counts()
def loan_group_age(row):
    age = row['dob_years']
    
    if age == 0:
        return 'No age specified'
    if age <= 17:
        return 'Adolescent'
    if age <=65:
        return 'Adult'
    if age > 66:
        return 'Elderly'
    
    return 'age unidentified'


bank_loan['age_group'] = bank_loan.apply(loan_group_age, axis=1)
bank_loan['age_group'].value_counts()


# Method to group loan type by income level
bank_loan['total_income'].value_counts()
def loan_group_income(row):
    income = row['total_income']
    
    if income == 0:
        return 'No income specified'
    if income <= 20000:
        return 'Low-level income'
    elif income <= 50000:
        return 'Medium-level income'
    elif income > 50000:
        return 'High-level income'
    
    return 'income unidentified'

bank_loan['income_group'] = bank_loan.apply(loan_group_income, axis=1)
bank_loan['income_group'].value_counts()




Medium-level income    12835
Low-level income        7370
High-level income       1320
Name: income_group, dtype: int64

### Conclusion

The purpose column was uncategorized, due to the fact that many of the purposes were the same however because of the wording of the rows they were not grouped together. 

To fix this issue it was necessary to create a method that groups the purpose column into seperate individual purposes of the same type. For example, any row that had the word house or housing in it went into the house list. At the end of organizing the purpose column there was a list for housing, weddings, real estate, property, cars, schooling, and purpose unknown. 

By organizing the purpose data into their own specific list it is easier to use the purpose column as each purpose can be identified and counted.

To categorize the education column I seperated the data into three categories: K-12, College education, and Graduate education to get a better understanding of the various education levels. A vast majority of the sample has a K-12 education level. 

The gender column is adequately categorized between M for male and F for female. The is one value with the gender value of 'XNA'.

The age column has alot of different values, it will be easier to look at the data after categorizing it into subgroups. Nearly all of the people in this sample are in the adult age group and between the ages of 18-65.

The income column also has a lot of different values, to make the data easier to work with it was put into subgroups. If a person makes less than 20,000 they have a low-level. If a person makes between 20,000-50,000 they have a medium-level income. If a person makes more than 50,000 they have a high-level income.

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [29]:
data_pivot = bank_loan.pivot_table(index = 'children', columns ='debt', values = 'days_employed', aggfunc = 'count').fillna(0)

data_pivot.head()

children_with_debt = data_pivot.loc[:, 1]

children_without_debt = data_pivot.loc[:,0]

children_total = children_with_debt + children_without_debt

data_pivot['percentage with kids and debt'] = (children_with_debt / children_total * 100)

print(data_pivot)


debt            0       1  percentage with kids and debt
children                                                
0         13086.0  1063.0                       7.512898
1          4420.0   445.0                       9.146968
2          1929.0   202.0                       9.479118
3           303.0    27.0                       8.181818
4            37.0     4.0                       9.756098
5             9.0     0.0                       0.000000


### Conclusion

In conclusion, majority of the sample have zero children and have occured no debt. This shows a correlation to the amount of children and repaying a loan on time. People with no children may have extra money to spend on repaying a loan because they do not have to pay the expensive cost of parenthood. The data shows a higher percentage of people who have debt with just one child.

- Is there a relation between marital status and repaying a loan on time?

In [34]:
data_pivot = bank_loan.pivot_table(index = 'family_status', columns ='debt', values = 'days_employed', aggfunc = 'count').fillna(0)   

data_pivot.head()

family_with_debt = data_pivot.loc[:, 1]

family_without_debt = data_pivot.loc[:,0]

family_total = family_with_debt + family_without_debt

data_pivot['percentage of debt based on family status'] = (family_with_debt / family_total * 100)

print(data_pivot)





debt                   0    1  percentage of debt based on family status
family_status                                                           
civil partnership   3789  388                                   9.288963
divorced            1110   85                                   7.112971
married            11449  931                                   7.520194
unmarried           2539  274                                   9.740491
widow / widower      897   63                                   6.562500


### Conclusion

The highest percentage of debt is close to 10% in unmarried people. Following behind is people in civil partnerships at 9%. There are low percentages of defaulted customers who are married.This could possibly be due to the fact that there are two incomes in the relationship. Therefore, more money is available to pay back loans on time. Divorced and widowed people also have a low default rate this could be due to settlement money or life insurance money. 

- Is there a relation between income level and repaying a loan on time?

In [36]:
data_pivot = bank_loan.pivot_table(index = 'income_group', columns ='debt', values = 'days_employed', aggfunc = 'count').fillna(0)

data_pivot.head()

income_with_debt = data_pivot.loc[:, 1]

income_without_debt = data_pivot.loc[:,0]

income_total = income_with_debt + income_without_debt

data_pivot['percentage of debt based on income group'] = (income_with_debt / income_total * 100)

print(data_pivot)

debt                     0     1  percentage of debt based on income group
income_group                                                              
High-level income     1228    92                                  6.969697
Low-level income      6762   608                                  8.249661
Medium-level income  11794  1041                                  8.110635


### Conclusion

People who have a high-level income are more likely to have no debt at 7%. This conclusion makes sense because if they have a high-level income and are living slightly at or below their means they have excess money. This excess money can go to pay back loans on time. People who have a low-level income are more likely to have debt. This is due to the fact that they may not have excess money to pay back a loan on time if they need to purchase other essential items with their paycheck. 

- How do different loan purposes affect on-time repayment of the loan?

In [40]:
data_pivot = bank_loan.pivot_table(index = 'purpose_group', columns ='debt', values = 'days_employed', aggfunc = 'count').fillna(0)

data_pivot.head()

purpose_with_debt = data_pivot.loc[:, 1]

purpose_without_debt = data_pivot.loc[:,0]

purpose_total = purpose_with_debt + purpose_without_debt

data_pivot['percentage of debt based on purpose group'] = (purpose_with_debt / purpose_total * 100)

print(data_pivot)

debt                          0    1  \
purpose_group                          
car loan purpose           3912  403   
purpose unidentified        373   39   
real estate loan purpose  10058  782   
school loan purpose        3279  331   
wedding loan purpose       2162  186   

debt                      percentage of debt based on purpose group  
purpose_group                                                        
car loan purpose                                           9.339513  
purpose unidentified                                       9.466019  
real estate loan purpose                                   7.214022  
school loan purpose                                        9.168975  
wedding loan purpose                                       7.921635  


### Conclusion

The highest number of people getting a loan is for a real estate purpposes. This includes any purpose that is related to housing, real estate, and property. However, the highest debt occurs with people who received a loan for purchasing a car. The second highest number of people with debt belong to the group that received a loan to help pay for school.

### Step 4. General conclusion

Based on the data, there are many factors that contribute to people paying back a loan on time. For example, it looks as if people in a relationship with a significant other are less likely to default on a loan and owe debt. The main reason I believe that people in relationships are able to pay back a loan on time is because they are in a two income household with more disposable income. This also explains why people with a high-level income also have less debt. Another deciding factor is the amount of children that a person has because according to the data the less children the less debt. This could be due to the fact that there is extra income available that is not spent on child care to pay loans back on time. In order to determine who will be a viable candidate the different factors should be compared together to get a better overall picture. My analysis of the data concludes that the most viable candidate to repay a loan on time would be a married person with no kids and a high-level income. 